In [ ]:
import pandas as pd
import sys
import os
import clickhouse_connect
import dotenv
import logging
import pandas as pd
import logging
from datetime import time
from collections import defaultdict
dotenv.load_dotenv()


In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('options_backtest.log')
    ]
)

In [ ]:
host = os.getenv('CLICKHOUSE_HOST', 'localhost')
port = int(os.getenv('CLICKHOUSE_PORT', 8123))
username = os.getenv('CLICKHOUSE_USER', 'default')
password = os.getenv('CLICKHOUSE_PASSWORD', '')

client = clickhouse_connect.get_client(
host=host,
port=port,
username=username,
password=password
)


In [ ]:
pricesdf = pd.DataFrame(columns=['strike_price', 'option_type', 'current_datetime', 'expiry_date', 'open'])

In [ ]:
def get_option_price(strike_price, option_type, current_datetime, expiry_date):
    global pricesdf  # Use the global DataFrame
    # Add 5 minutes to the current datetime
    current_datetime = current_datetime + pd.Timedelta(minutes=5)
    
    try:
        # Format datetime for ClickHouse - remove timezone info and use standard format
        if isinstance(current_datetime, pd.Timestamp):
            formatted_datetime = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
        else:
            formatted_datetime = pd.to_datetime(current_datetime).strftime('%Y-%m-%d %H:%M:%S')
            
        query = f"""
        SELECT open 
        FROM minute_data.options 
        WHERE datetime = '{formatted_datetime}' 
          AND strike_price = {strike_price} 
          AND expiry_date = '{expiry_date}' 
          AND option_type = '{option_type}'
        """
        result = client.query_df(query)
        # logging.debug(f"result:{result}, TIME:{formatted_datetime}")
        if not result.empty:
            price = result['open'].iloc[0]
            # Create a new row DataFrame to concatenate
            new_row = pd.DataFrame([{
                'strike_price': strike_price,
                'option_type': option_type,
                'current_datetime': formatted_datetime,
                'expiry_date': expiry_date,
                'open': price
            }])
            pricesdf = pd.concat([pricesdf, new_row], ignore_index=True)
            return price
        else:
            logging.warning(f"No close price found for {strike_price}, {option_type} at {formatted_datetime} for expiry {expiry_date}")
            # Use the last found price for the same strike, option type, and expiry date
            fallback_df = pricesdf[
                (pricesdf['strike_price'] == strike_price) &
                (pricesdf['option_type'] == option_type) &
                (pricesdf['expiry_date'] == expiry_date)
            ]
            if not fallback_df.empty:
                last_record = fallback_df.iloc[-1]
                fallback_price = last_record['open']
                fallback_datetime = last_record['current_datetime']
                logging.warning(f"Using fallback price {fallback_price} found at {fallback_datetime}")
                return fallback_price
            else:
                return False
    except Exception as e:
        logging.error(f"Error while fetching close price for {strike_price} at {formatted_datetime}")
        logging.error(e)
        return False


In [ ]:
OPTIONS_RATE = 11300
DAILY_INTEREST = 32.87  # Daily interest cost

def calculate_options_expenses(buy_quantity: float, buy_rate: float, sell_quantity: float, sell_rate: float) -> dict:
    """
    Calculate expenses for options trades with detailed breakdown
    
    Args:
        buy_quantity: Quantity bought (in number of contracts)
        buy_rate: Price at which bought
        sell_quantity: Quantity sold (in number of contracts)
        sell_rate: Price at which sold
        
    Returns:
        Dictionary with trade values and expense details
    """
    buy_value = buy_quantity * buy_rate
    sell_value = sell_quantity * sell_rate
    total_value = buy_value + sell_value
    
    # Convert to crores for calculation
    turnover_in_crores = total_value / 10000000
    expenses = turnover_in_crores * OPTIONS_RATE
    
    return {
        "buy_value": buy_value,
        "sell_value": sell_value,
        "total_value": total_value,
        "expenses": expenses
    }

In [ ]:
# --- PARAMETERS ---
STRIKE_DIFF    = 50           # Round strikes to nearest 50
LOT_SIZE       = 75           # Contract size
START_TIME     = time(9, 30)  # Trading start
END_TIME       = time(15, 15) # End-of-day square-off
DAILY_INTEREST = 32.87        # Fixed daily interest cost
BASE_CAPITAL   = 200000       # Base capital for M2M calculations
MAX_STOP_LOSS   = 1.5         # Max stop-loss in percentage of base capital (1.5%)
TARGET_PROFIT   = 3.0         # Profit target in percentage of base capital (3%)
TRAILING_START  = 1.5         # Start trailing at this profit percentage of base capital (1.5%)

# Date range for backtesting - USER CAN CHANGE THESE
START_DATE = '2021-01-01'
END_DATE = '2025-06-30'

def run_pattern_backtest(df: pd.DataFrame, pattern_col: str) -> pd.DataFrame:
    """
    Backtest strategy for a specific pattern: sell CALL on -1 signal, sell PUT on 1 signal,
    with M2M-based profit/loss calculations relative to base capital of ₹200,000.
    
    Args:
        df: DataFrame with datetime, close, closest_expiry, and pattern columns
        pattern_col: Column name for the pattern to backtest (e.g., 'pattern_0')
        
    Returns:
        DataFrame with per-trade metrics
    """
    # Prepare DataFrame
    data = df.copy()
    data['datetime'] = pd.to_datetime(data['datetime'])

    
    # Filter by date range
    data = data[(data['datetime'] >= START_DATE) & (data['datetime'] <= END_DATE)].copy()
    
    data['expiry_date'] = pd.to_datetime(data['closest_expiry']).dt.date
    data['strike_price'] = (data['close'] / STRIKE_DIFF).round() * STRIKE_DIFF
    data['strike_price'] = data['strike_price'].astype(int)
    
    # Convert pattern signals: 1 = Buy signal, -1 = Sell signal, 0 = No signal
    data['signal'] = data[pattern_col].map({1: 'Buy', -1: 'Sell', 0: None})

    # Tracking variables
    position = None  # 'CE' or 'PE'
    entry_price = 0.0
    realized_pnl = 0.0
    highest_m2m = 0.0  # Track highest M2M value for trailing
    trailing_enabled = False
    entry_time = None
    
    # Calculate M2M thresholds based on base capital
    max_loss_amount = BASE_CAPITAL * (MAX_STOP_LOSS / 100)  # 1.5% of 200,000 = 3,000
    target_profit_amount = BASE_CAPITAL * (TARGET_PROFIT / 100)  # 3% of 200,000 = 6,000
    trailing_start_amount = BASE_CAPITAL * (TRAILING_START / 100)  # 1.5% of 200,000 = 3,000
    
    # Daily trade tracking
    daily_trades = defaultdict(int)
    daily_interest_applied = set()

    metrics = []
    logging.info(f"Starting backtest for {pattern_col} (Date range: {START_DATE} to {END_DATE})")
    logging.info(f"Base Capital: ₹{BASE_CAPITAL:,}, Max Loss: ₹{max_loss_amount:,.0f}, Target: ₹{target_profit_amount:,.0f}, Trailing Start: ₹{trailing_start_amount:,.0f}")

    for idx, row in data.iterrows():
        dt = row.datetime
        sig = row.signal
        strike = row.strike_price
        exp = row.expiry_date
        current_date = dt.date()

        # Skip outside trading hours
        if not (START_TIME <= dt.time() <= END_TIME):
            continue
        
        # Manage open position: M2M, SL and trailing logic
        if position:
            current_price = get_option_price(strike, position, dt, exp)
            if current_price is False:
                continue
                
            # Calculate M2M (profit/loss in absolute terms)
            m2m = (entry_price - current_price) * LOT_SIZE
            cum_m2m = realized_pnl + m2m

            # Update highest M2M for trailing
            if m2m > highest_m2m:
                highest_m2m = m2m
                
            # Enable trailing once we reach trailing start amount (₹3,000)
            if not trailing_enabled and m2m >= trailing_start_amount:
                trailing_enabled = True
                logging.info(f"{dt} *** M2M ₹{m2m:.0f} >= ₹{trailing_start_amount:.0f}, enabling trailing SL")

            # Check max stop-loss (₹3,000 loss)
            if m2m <= -max_loss_amount:
                exit_time = dt
                exit_price = current_price
                logging.info(f"{dt} !!! Max SL hit: M2M ₹{m2m:.0f} <= -₹{max_loss_amount:.0f}, exiting {position}")
                
                # Calculate P&L and expenses
                gross_pnl = m2m
                realized_pnl += gross_pnl
                
                trade_expenses = calculate_options_expenses(
                    LOT_SIZE, entry_price, LOT_SIZE, exit_price
                )["expenses"]
                
                trade_date = exit_time.date()
                daily_trades[trade_date] += 1
                
                interest_cost = 0
                if trade_date not in daily_interest_applied:
                    interest_cost = DAILY_INTEREST
                    daily_interest_applied.add(trade_date)
                
                net_pnl = gross_pnl - trade_expenses - interest_cost
                
                metrics.append({
                    'pattern': pattern_col,
                    'strike_price': strike,
                    'option_type': position,
                    'entry_time': entry_time,
                    'entry_price': entry_price,
                    'exit_time': exit_time,
                    'exit_price': exit_price,
                    'exit_reason': 'Max SL',
                    'm2m': m2m,
                    'trade_date': trade_date,
                    'gross_pnl': gross_pnl,
                    'expenses': trade_expenses,
                    'interest': interest_cost,
                    'net_pnl': net_pnl,
                    'daily_trades': daily_trades[trade_date]
                })
                
                # Reset state
                position = None
                highest_m2m = 0.0
                trailing_enabled = False
                continue

            # Check profit target (₹6,000 profit)
            if m2m >= target_profit_amount:
                exit_time = dt
                exit_price = current_price
                logging.info(f"{dt} !!! Profit target hit: M2M ₹{m2m:.0f} >= ₹{target_profit_amount:.0f}, exiting {position}")
                
                gross_pnl = m2m
                realized_pnl += gross_pnl
                
                trade_expenses = calculate_options_expenses(
                    LOT_SIZE, entry_price, LOT_SIZE, exit_price
                )["expenses"]
                
                trade_date = exit_time.date()
                daily_trades[trade_date] += 1
                
                interest_cost = 0
                if trade_date not in daily_interest_applied:
                    interest_cost = DAILY_INTEREST
                    daily_interest_applied.add(trade_date)
                
                net_pnl = gross_pnl - trade_expenses - interest_cost
                
                metrics.append({
                    'pattern': pattern_col,
                    'strike_price': strike,
                    'option_type': position,
                    'entry_time': entry_time,
                    'entry_price': entry_price,
                    'exit_time': exit_time,
                    'exit_price': exit_price,
                    'exit_reason': 'Target Hit',
                    'm2m': m2m,
                    'trade_date': trade_date,
                    'gross_pnl': gross_pnl,
                    'expenses': trade_expenses,
                    'interest': interest_cost,
                    'net_pnl': net_pnl,
                    'daily_trades': daily_trades[trade_date]
                })
                
                position = None
                highest_m2m = 0.0
                trailing_enabled = False
                continue

            # Check trailing stop-loss
            if trailing_enabled:
                # Trail with highest M2M: if highest was ₹4,000, new SL at ₹1,000 (₹4,000 - ₹3,000)
                trailing_sl_amount = max(0.0, highest_m2m - trailing_start_amount)
                
                if m2m <= trailing_sl_amount:
                    exit_time = dt
                    exit_price = current_price
                    logging.info(f"{dt} !!! Trailing SL hit: M2M ₹{m2m:.0f} <= ₹{trailing_sl_amount:.0f} (highest was ₹{highest_m2m:.0f}), exiting {position}")
                    
                    gross_pnl = m2m
                    realized_pnl += gross_pnl
                    
                    trade_expenses = calculate_options_expenses(
                        LOT_SIZE, entry_price, LOT_SIZE, exit_price
                    )["expenses"]
                    
                    trade_date = exit_time.date()
                    daily_trades[trade_date] += 1
                    
                    interest_cost = 0
                    if trade_date not in daily_interest_applied:
                        interest_cost = DAILY_INTEREST
                        daily_interest_applied.add(trade_date)
                    
                    net_pnl = gross_pnl - trade_expenses - interest_cost
                    
                    metrics.append({
                        'pattern': pattern_col,
                        'strike_price': strike,
                        'option_type': position,
                        'entry_time': entry_time,
                        'entry_price': entry_price,
                        'exit_time': exit_time,
                        'exit_price': exit_price,
                        'exit_reason': 'Trailing SL',
                        'm2m': m2m,
                        'trade_date': trade_date,
                        'gross_pnl': gross_pnl,
                        'expenses': trade_expenses,
                        'interest': interest_cost,
                        'net_pnl': net_pnl,
                        'daily_trades': daily_trades[trade_date]
                    })
                    
                    position = None
                    highest_m2m = 0.0
                    trailing_enabled = False
                    continue

        # EOD square-off
        if position and dt.time() >= END_TIME:
            exit_time = dt
            exit_price = get_option_price(strike, position, dt, exp)
            if exit_price is not False:
                m2m = (entry_price - exit_price) * LOT_SIZE
                logging.info(f"{dt} → EOD square-off {position} @ {exit_price:.2f}, M2M: ₹{m2m:.0f}")
                
                gross_pnl = m2m
                realized_pnl += gross_pnl
                
                trade_expenses = calculate_options_expenses(
                    LOT_SIZE, entry_price, LOT_SIZE, exit_price
                )["expenses"]
                
                trade_date = exit_time.date()
                daily_trades[trade_date] += 1
                
                interest_cost = 0
                if trade_date not in daily_interest_applied:
                    interest_cost = DAILY_INTEREST
                    daily_interest_applied.add(trade_date)
                
                net_pnl = gross_pnl - trade_expenses - interest_cost
                
                metrics.append({
                    'pattern': pattern_col,
                    'strike_price': strike,
                    'option_type': position,
                    'entry_time': entry_time,
                    'entry_price': entry_price,
                    'exit_time': exit_time,
                    'exit_price': exit_price,
                    'exit_reason': 'EOD',
                    'm2m': m2m,
                    'trade_date': trade_date,
                    'gross_pnl': gross_pnl,
                    'expenses': trade_expenses,
                    'interest': interest_cost,
                    'net_pnl': net_pnl,
                    'daily_trades': daily_trades[trade_date]
                })
                
            position = None
            highest_m2m = 0.0
            trailing_enabled = False
            continue

        # Entry logic on signals
        if sig == 'Buy' and position != 'CE':
            # Exit short PUT if active
            if position == 'PE':
                exit_time = dt
                exit_price = get_option_price(strike, 'PE', dt, exp)
                if exit_price is not False:
                    m2m = (entry_price - exit_price) * LOT_SIZE
                    logging.info(f"{dt} → Exit short PUT @ {exit_price:.2f}, M2M: ₹{m2m:.0f}")
                    
                    gross_pnl = m2m
                    realized_pnl += gross_pnl
                    
                    trade_expenses = calculate_options_expenses(
                        LOT_SIZE, entry_price, LOT_SIZE, exit_price
                    )["expenses"]
                    
                    trade_date = exit_time.date()
                    daily_trades[trade_date] += 1
                    
                    interest_cost = 0
                    if trade_date not in daily_interest_applied:
                        interest_cost = DAILY_INTEREST
                        daily_interest_applied.add(trade_date)
                    
                    net_pnl = gross_pnl - trade_expenses - interest_cost
                    
                    metrics.append({
                        'pattern': pattern_col,
                        'strike_price': strike,
                        'option_type': 'PE',
                        'entry_time': entry_time,
                        'entry_price': entry_price,
                        'exit_time': exit_time,
                        'exit_price': exit_price,
                        'exit_reason': 'Signal Change',
                        'm2m': m2m,
                        'trade_date': trade_date,
                        'gross_pnl': gross_pnl,
                        'expenses': trade_expenses,
                        'interest': interest_cost,
                        'net_pnl': net_pnl,
                        'daily_trades': daily_trades[trade_date]
                    })
                
            # Enter short CALL
            entry_time = dt
            entry_price = get_option_price(strike, 'CE', dt, exp)
            if entry_price is not False:
                position = 'CE'
                highest_m2m = 0.0
                trailing_enabled = False
                logging.info(f"{dt} → Enter short CALL @ {entry_price:.2f}")

        elif sig == 'Sell' and position != 'PE':
            # Exit short CALL if active
            if position == 'CE':
                exit_time = dt
                exit_price = get_option_price(strike, 'CE', dt, exp)
                if exit_price is not False:
                    m2m = (entry_price - exit_price) * LOT_SIZE
                    logging.info(f"{dt} → Exit short CALL @ {exit_price:.2f}, M2M: ₹{m2m:.0f}")
                    
                    gross_pnl = m2m
                    realized_pnl += gross_pnl
                    
                    trade_expenses = calculate_options_expenses(
                        LOT_SIZE, entry_price, LOT_SIZE, exit_price
                    )["expenses"]
                    
                    trade_date = exit_time.date()
                    daily_trades[trade_date] += 1
                    
                    interest_cost = 0
                    if trade_date not in daily_interest_applied:
                        interest_cost = DAILY_INTEREST
                        daily_interest_applied.add(trade_date)
                    
                    net_pnl = gross_pnl - trade_expenses - interest_cost
                    
                    metrics.append({
                        'pattern': pattern_col,
                        'strike_price': strike,
                        'option_type': 'CE',
                        'entry_time': entry_time,
                        'entry_price': entry_price,
                        'exit_time': exit_time,
                        'exit_price': exit_price,
                        'exit_reason': 'Signal Change',
                        'm2m': m2m,
                        'trade_date': trade_date,
                        'gross_pnl': gross_pnl,
                        'expenses': trade_expenses,
                        'interest': interest_cost,
                        'net_pnl': net_pnl,
                        'daily_trades': daily_trades[trade_date]
                    })
                
            # Enter short PUT
            entry_time = dt
            entry_price = get_option_price(strike, 'PE', dt, exp)
            if entry_price is not False:
                position = 'PE'
                highest_m2m = 0.0
                trailing_enabled = False
                logging.info(f"{dt} → Enter short PUT @ {entry_price:.2f}")

    # Create metrics DataFrame
    metrics_df = pd.DataFrame(metrics)
    
    if not metrics_df.empty:
        total_trades = len(metrics_df)
        total_gross_pnl = metrics_df['gross_pnl'].sum()
        total_expenses = metrics_df['expenses'].sum()
        total_interest = metrics_df['interest'].sum()
        total_net_pnl = metrics_df['net_pnl'].sum()
        
        logging.info(f"Backtest complete for {pattern_col}: {total_trades} trades")
        logging.info(f"Total Gross P&L: ₹{total_gross_pnl:.2f}")
        logging.info(f"Total Expenses: ₹{total_expenses:.2f}")
        logging.info(f"Total Interest: ₹{total_interest:.2f}")
        logging.info(f"Total Net P&L: ₹{total_net_pnl:.2f}")
        
        metrics_df['total_trades'] = total_trades
        
    return metrics_df

def run_all_patterns_backtest(csv_file_path: str):
    """
    Run backtest for all patterns (pattern_0 to pattern_9) and save results to separate CSV files.
    
    Args:
        csv_file_path: Path to the signals CSV file
    """
    # Load data
    logging.info(f"Loading data from {csv_file_path}")
    df = pd.read_csv(csv_file_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['closest_expiry'] = pd.to_datetime(df['closest_expiry']).dt.date



    #dates to drop
    dates_to_drop = ['2020-01-01', '2024-11-22']
    #drop these drom datetime column
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df[~df['datetime'].dt.date.isin([pd.to_datetime(date).date() for date in dates_to_drop])]


    # Get all pattern columns
    pattern_columns = [col for col in df.columns if col.startswith('pattern_')]
    logging.info(f"Found pattern columns: {pattern_columns}")
    
    # Run backtest for each pattern
    for pattern_col in pattern_columns:
        logging.info(f"\n{'='*50}")
        logging.info(f"Running backtest for {pattern_col}")
        logging.info(f"{'='*50}")
        
        try:
            # Reset global prices dataframe for each pattern
            global pricesdf
            pricesdf = pd.DataFrame(columns=['strike_price', 'option_type', 'current_datetime', 'expiry_date', 'open'])
            
            # Run backtest
            results_df = run_pattern_backtest(df, pattern_col)
            
            if not results_df.empty:
                # Save results to CSV
                output_file = f'/home/algolinux/Documents/aviral/Ichimoku-ADX-Wilder/backtesting/results/{pattern_col}_backtest_results.csv'
                results_df.to_csv(output_file, index=False)
                logging.info(f"Results saved to {output_file}")
                
                # Print summary
                win_trades = results_df[results_df['net_pnl'] > 0]
                loss_trades = results_df[results_df['net_pnl'] <= 0]
                win_rate = len(win_trades) / len(results_df) * 100 if len(results_df) > 0 else 0
                
                logging.info(f"\n{pattern_col} Summary:")
                logging.info(f"Total Trades: {len(results_df)}")
                logging.info(f"Winning Trades: {len(win_trades)}")
                logging.info(f"Losing Trades: {len(loss_trades)}")
                logging.info(f"Win Rate: {win_rate:.2f}%")
                logging.info(f"Total Net P&L: ₹{results_df['net_pnl'].sum():.2f}")
                logging.info(f"Average P&L per Trade: ₹{results_df['net_pnl'].mean():.2f}")
                
            else:
                logging.warning(f"No trades found for {pattern_col}")
                
        except Exception as e:
            logging.error(f"Error processing {pattern_col}: {str(e)}")
            continue
    
    logging.info(f"\n{'='*50}")
    logging.info("All pattern backtests completed!")
    logging.info(f"{'='*50}")

# Example usage:
# To run backtest for all patterns, uncomment the line below and run the cell:
# run_all_patterns_backtest('/home/algolinux/Documents/aviral/Ichimoku-ADX-Wilder/backtesting/data/ichimoku_adx_wilder_signals.csv')

In [ ]:
# Run backtest for all patterns
# You can modify the START_DATE and END_DATE variables above before running this

# Create results directory if it doesn't exist
import os
results_dir = '/home/algolinux/Documents/aviral/Ichimoku-ADX-Wilder/backtesting/results'
os.makedirs(results_dir, exist_ok=True)

# Run the backtest for all patterns
csv_file_path = '/home/algolinux/Documents/aviral/Ichimoku-ADX-Wilder/backtesting/data/ichimoku_adx_wilder_signals.csv'
run_all_patterns_backtest(csv_file_path)